### 클릭한 뉴스들 중 고유한 뉴스 ID 정리

In [13]:
import pandas as pd

# =========================
# 설정
# =========================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'
OUT_CLICKED_IDS = 'clicked_newsIds_global.csv'   # 다음 단계에서 재사용할 파일

# =========================
# 1) 데이터 불러오기 (헤더 없음)
# =========================
train = pd.read_csv(PATH_TRAIN, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
dev   = pd.read_csv(PATH_DEV,   sep='\t', names=BEHAVIOR_COLUMNS, header=None)

# =========================
# 2) train+dev 합치기
# =========================
behaviors = pd.concat([train, dev], ignore_index=True)

# =========================
# 3) 클릭된 뉴스 고유 ID 수집 (초보자용 for문)
#    - Impressions를 공백으로 나눔
#    - "newsId-label"에서 label이 '1'인 것만 수집
#    - rsplit('-', 1)로 뒤에서 한 번만 나눠 안전 파싱
# =========================
clicked_ids = set()

for _, row in behaviors.iterrows():
    imps = str(row['Impressions']).split()
    for tok in imps:
        if '-' not in tok:
            continue
        news_id, label = tok.rsplit('-', 1)
        if label.strip() == '1':
            clicked_ids.add(news_id.strip())

# =========================
# 4) DataFrame으로 변환 + 저장
# =========================
clicked_list = sorted(clicked_ids)  # 재현성 위해 정렬(선택)
clicked_df = pd.DataFrame({'news_id': clicked_list})

print("고유 클릭 뉴스 개수:", len(clicked_df))
print(clicked_df.head())

# 다음 단계에서 재사용할 수 있도록 저장
clicked_df.to_csv(OUT_CLICKED_IDS, index=False)
print(f"저장 완료 → {OUT_CLICKED_IDS}")


고유 클릭 뉴스 개수: 9100
  news_id
0  N10032
1  N10050
2  N10051
3  N10056
4  N10057
저장 완료 → clicked_newsIds_global.csv


In [14]:
import pandas as pd
from datetime import timedelta

# =========================
# 설정
# =========================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'
PATH_CLICKED_IDS = 'clicked_newsIds_global.csv'     # 1단계에서 만든 파일 (col: news_id)
OUT_LIFESPAN = 'news_times_global.csv'          # 전체 lifespan 저장 파일

# =========================
# 1) 데이터 불러오기 (헤더 없음 주의)
# =========================
train = pd.read_csv(PATH_TRAIN, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
dev   = pd.read_csv(PATH_DEV,   sep='\t', names=BEHAVIOR_COLUMNS, header=None)

# 시간 파싱(미리 한 번에) → NaT 허용
train['Time'] = pd.to_datetime(train['Time'], errors='coerce')
dev['Time']   = pd.to_datetime(dev['Time'],   errors='coerce')

# 합치기
behaviors = pd.concat([train, dev], ignore_index=True)

# =========================
# 2) 9,100 허용 집합(클릭 고유 뉴스) 불러오기
# =========================
allow_df = pd.read_csv(PATH_CLICKED_IDS)
allowed_ids = set(str(x).strip() for x in allow_df['news_id'].tolist())

# =========================
# 3) 각 뉴스의 '최초 노출 시각' 찾기 (라벨 0/1 무관, 허용 집합에 한정)
#    - impressions를 공백으로 나눔
#    - "newsId-label"은 rsplit('-', 1)로 안전 파싱
#    - allowed_ids에 속하는 뉴스만 고려
#    - 가장 이른 시각을 publish_time으로 저장
# =========================
first_time = {}  # dict: news_id -> 가장 이른 datetime

for _, row in behaviors.iterrows():
    t = row['Time']
    if pd.isna(t):
        continue
    tokens = str(row['Impressions']).split()
    for tok in tokens:
        if '-' not in tok:
            continue
        news_id, _ = tok.rsplit('-', 1)
        news_id = news_id.strip()

        # 허용 집합(= 9,100) 안에서만 처리
        if news_id not in allowed_ids:
            continue

        # 최초 등장 시각 갱신
        if news_id not in first_time:
            first_time[news_id] = t
        else:
            if t < first_time[news_id]:
                first_time[news_id] = t

# =========================
# 4) 표로 만들고 lifespan_end = publish_time + 36시간 계산
# =========================
rows = []
for nid, pub_t in first_time.items():
    rows.append((nid, pub_t, pub_t + timedelta(hours=36)))

news_lifespan_df = pd.DataFrame(rows, columns=['news_id', 'publish_time', 'lifespan_end'])

# 정렬은 선택(보기 편하게)
news_lifespan_df = news_lifespan_df.sort_values('publish_time').reset_index(drop=True)

print(news_lifespan_df.head())
print("총 개수:", len(news_lifespan_df))

# (참고) 허용 집합에 있었지만 behaviors에서 시간이 잡히지 않은 뉴스가 있는지 체크
missing_ids = sorted(list(allowed_ids - set(first_time.keys())))
print("허용 집합 중 publish_time을 찾지 못한 개수:", len(missing_ids))
# 필요하면 아래 주석 해제해서 어떤 ID들인지 확인
# print(missing_ids[:20])

# =========================
# 5) 저장 (다음 단계에서 재사용)
# =========================
news_lifespan_df.to_csv(OUT_LIFESPAN, index=False)
print(f"저장 완료 → {OUT_LIFESPAN}")


  news_id        publish_time        lifespan_end
0  N16560 2019-11-09 00:00:19 2019-11-10 12:00:19
1  N50329 2019-11-09 00:00:19 2019-11-10 12:00:19
2  N15134 2019-11-09 00:00:19 2019-11-10 12:00:19
3  N37108 2019-11-09 00:00:19 2019-11-10 12:00:19
4  N58075 2019-11-09 00:00:19 2019-11-10 12:00:19
총 개수: 9100
허용 집합 중 publish_time을 찾지 못한 개수: 0
저장 완료 → news_times_global.csv


In [15]:
import pandas as pd

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 헤더 없음 주의
train = pd.read_csv('download/MINDsmall_train/behaviors.tsv', sep='\t',
                    names=BEHAVIOR_COLUMNS, header=None)

def count_pos_in_row(imps_str):
    """한 행의 Impressions에서 -1(클릭) 개수 세기 (초보자용 안전 파싱)"""
    if pd.isna(imps_str):
        return 0
    cnt = 0
    for token in str(imps_str).split():
        if '-' not in token:
            continue
        news_id, label = token.rsplit('-', 1)
        if label.strip() == '1':
            cnt += 1
    return cnt

train['pos_cnt'] = train['Impressions'].apply(count_pos_in_row)

rows_with_multi_pos = (train['pos_cnt'] >= 2).sum()
print("전체 train 행 개수:", len(train))
print("한 행에 impression-1(클릭)이 2개 이상인 행 개수:", rows_with_multi_pos)
print(len(train)-rows_with_multi_pos)


전체 train 행 개수: 156965
한 행에 impression-1(클릭)이 2개 이상인 행 개수: 43077
113888


In [16]:
import pandas as pd

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'
PATH_TRAIN_LIFE = 'train_lifespan.csv'  # ['news_id','publish_time','lifespan_end']
PATH_DEV_LIFE   = 'dev_lifespan.csv'

def count_allowed_pos(imps_str, allowed_ids):
    """허용 집합(=lifespan news_id) 기준으로 -1(클릭) 개수 세기"""
    if pd.isna(imps_str):
        return 0
    cnt = 0
    for tok in str(imps_str).split():
        if '-' not in tok:
            continue
        nid, lab = tok.rsplit('-', 1)
        if (lab.strip() == '1') and (nid.strip() in allowed_ids):
            cnt += 1
    return cnt

def analyze_split(beh_path, life_path, split_name):
    # 1) 데이터 로드 + 시간 파싱
    df = pd.read_csv(beh_path, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

    # 2) 허용 집합(= 해당 split lifespan의 news_id)
    life_df = pd.read_csv(life_path)
    allowed_ids = set(str(x).strip() for x in life_df['news_id'].dropna().tolist())

    # 3) 4단계 규칙에 맞춰 '예상 변경 후 행수'와 분포 계산
    orig_rows = len(df)
    new_rows = 0

    rows_with_multi = 0
    sum_pos_in_multi = 0
    dist = {}  # {양성개수: 행 수}

    rows_time_nat = 0
    rows_with_at_least_one_allowed_pos = 0

    for _, row in df.iterrows():
        pos_cnt_allowed = count_allowed_pos(row['Impressions'], allowed_ids)
        is_nat = pd.isna(row['Time'])

        # 분포(허용 집합 기준)
        if pos_cnt_allowed >= 2:
            rows_with_multi += 1
            sum_pos_in_multi += pos_cnt_allowed
            dist[pos_cnt_allowed] = dist.get(pos_cnt_allowed, 0) + 1

        # 4단계 확장 규칙과 동일하게 '예상 새 행수' 누적
        if (not is_nat) and (pos_cnt_allowed >= 1):
            new_rows += pos_cnt_allowed     # 양성 m개면 m행
            rows_with_at_least_one_allowed_pos += 1
        else:
            new_rows += 1                   # NaT이거나 양성 0개면 1행 유지
            if is_nat:
                rows_time_nat += 1

    delta = new_rows - orig_rows

    # 4) 출력 (split 별)
    print(f"\n===== {split_name} =====")
    print(f"원본 행수: {orig_rows:,}  →  예상 변경 후: {new_rows:,}  (Δ {delta:+,})")
    print(f"- Time이 NaT인 행 수: {rows_time_nat:,}")
    print(f"- 허용집합 기준 -1이 2개 이상인 행 수: {rows_with_multi:,}")
    print(f"- 그 행들에서의 -1 총합: {sum_pos_in_multi:,}")

    print("\n[-1 개수 분포 (허용집합 기준)]")
    for k in sorted(dist):
        print(f"{k}개 -1: {dist[k]}행")

# 실행
analyze_split(PATH_TRAIN, PATH_TRAIN_LIFE, 'train')
analyze_split(PATH_DEV,   PATH_DEV_LIFE,   'dev')



===== train =====
원본 행수: 156,965  →  예상 변경 후: 236,344  (Δ +79,379)
- Time이 NaT인 행 수: 0
- 허용집합 기준 -1이 2개 이상인 행 수: 43,077
- 그 행들에서의 -1 총합: 122,456

[-1 개수 분포 (허용집합 기준)]
2개 -1: 25571행
3개 -1: 9263행
4개 -1: 3975행
5개 -1: 1957행
6개 -1: 942행
7개 -1: 515행
8개 -1: 296행
9개 -1: 198행
10개 -1: 117행
11개 -1: 81행
12개 -1: 46행
13개 -1: 38행
14개 -1: 22행
15개 -1: 17행
16개 -1: 10행
17개 -1: 6행
18개 -1: 9행
19개 -1: 2행
20개 -1: 1행
21개 -1: 2행
22개 -1: 1행
23개 -1: 1행
24개 -1: 1행
25개 -1: 1행
26개 -1: 2행
27개 -1: 1행
31개 -1: 1행
35개 -1: 1행

===== dev =====
원본 행수: 73,152  →  예상 변경 후: 111,383  (Δ +38,231)
- Time이 NaT인 행 수: 0
- 허용집합 기준 -1이 2개 이상인 행 수: 21,085
- 그 행들에서의 -1 총합: 59,316

[-1 개수 분포 (허용집합 기준)]
2개 -1: 12707행
3개 -1: 4443행
4개 -1: 1911행
5개 -1: 932행
6개 -1: 426행
7개 -1: 268행
8개 -1: 166행
9개 -1: 83행
10개 -1: 61행
11개 -1: 34행
12개 -1: 17행
13개 -1: 11행
14개 -1: 5행
15개 -1: 5행
16개 -1: 6행
17개 -1: 3행
18개 -1: 2행
19개 -1: 2행
20개 -1: 1행
21개 -1: 1행
24개 -1: 1행


In [17]:
import os
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

# =========================
# 설정
# =========================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'
PATH_NEWS_GLOBAL = 'news_times_global.csv'  # 허용 news id를 한 번에 사용

# =========================
# 허용 집합: news_times_global에서 news_id 컬럼 자동 감지
# =========================
assert os.path.exists(PATH_NEWS_GLOBAL), f"파일 없음: {PATH_NEWS_GLOBAL}"
ng = pd.read_csv(PATH_NEWS_GLOBAL, parse_dates=['publish_time','lifespan_end'])
cand_cols = [c for c in ng.columns if c.lower() in ('news_id', 'newsid', 'nid')]
if not cand_cols:
    # 흔한 케이스(NewsID/news_id)도 커버
    if 'NewsID' in ng.columns:
        cand_cols = ['NewsID']
    elif 'news_id' in ng.columns:
        cand_cols = ['news_id']
    else:
        raise ValueError(f"{PATH_NEWS_GLOBAL}에 news_id 컬럼이 필요합니다. 실제 컬럼: {list(ng.columns)}")
NEWS_COL = cand_cols[0]
ALLOWED_IDS = set(ng[NEWS_COL].astype(str).str.strip())
print(f"[허용 집합] news_times_global: rows={len(ng):,}, allowed_ids={len(ALLOWED_IDS):,}, col='{NEWS_COL}'")

# =========================
# 유틸
# =========================
def count_allowed_pos(imps_str, allowed_ids):
    """허용 집합 기준으로 -1(클릭) 개수 세기"""
    if pd.isna(imps_str):
        return 0
    cnt = 0
    for tok in str(imps_str).split():
        if '-' not in tok:
            continue
        nid, lab = tok.rsplit('-', 1)
        if (lab.strip() == '1') and (nid.strip() in allowed_ids):
            cnt += 1
    return cnt

def analyze_split(beh_path, split_name, allowed_ids, print_top_k=15):
    assert os.path.exists(beh_path), f"behaviors 파일 없음: {beh_path}"
    df = pd.read_csv(beh_path, sep='\t', names=BEHAVIOR_COLUMNS, header=None)

    # 로깅
    print(f"\n===== {split_name} : 파일 로드 =====")
    print(f"- behaviors: {beh_path} rows={len(df):,}")

    # Time 파싱
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    nat_before = df['Time'].isna().sum()
    print(f"- Time NaT(파싱 실패) 건수: {nat_before:,}")

    # 통계
    orig_rows = len(df)
    new_rows = 0
    rows_with_multi = 0
    sum_pos_in_multi = 0
    freq = defaultdict(int)
    rows_time_nat = 0
    rows_with_at_least_one_allowed_pos = 0

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"[{split_name}] scanning"):
        pos_cnt_allowed = count_allowed_pos(row['Impressions'], allowed_ids)
        is_nat = pd.isna(row['Time'])

        if pos_cnt_allowed >= 2:
            rows_with_multi += 1
            sum_pos_in_multi += pos_cnt_allowed
            freq[pos_cnt_allowed] += 1

        # 규칙: (Time 유효) & (허용집합 기준 양성>=1) → 양성 m개면 m행으로 분리
        if (not is_nat) and (pos_cnt_allowed >= 1):
            new_rows += pos_cnt_allowed
            rows_with_at_least_one_allowed_pos += 1
        else:
            new_rows += 1
            if is_nat:
                rows_time_nat += 1

    delta = new_rows - orig_rows

    # 출력
    print(f"\n===== {split_name} : 요약 (허용집합=news_times_global) =====")
    print(f"원본 행수: {orig_rows:,}  →  예상 변경 후: {new_rows:,}  (Δ {delta:+,})")
    print(f"- Time이 NaT인 행 수: {rows_time_nat:,}")
    print(f"- 허용집합 기준 -1이 2개 이상인 행 수: {rows_with_multi:,}")
    print(f"- 그 행들에서의 -1 총합: {sum_pos_in_multi:,}")
    print(f"- 허용집합 기준 -1≥1 인 행 수: {rows_with_at_least_one_allowed_pos:,}")

    if freq:
        print("\n[-1 개수 분포 (허용집합 기준) 상위]")
        for k in sorted(freq)[:print_top_k]:
            print(f"{k}개 -1: {freq[k]}행")
        if len(freq) > print_top_k:
            rest = len(freq) - print_top_k
            print(f"... (그 외 {rest}개 bin)")

# =========================
# 실행: news_times_global 한 벌로 train/dev 동시 분석
# =========================
analyze_split(PATH_TRAIN, 'train', ALLOWED_IDS)
analyze_split(PATH_DEV,   'dev',   ALLOWED_IDS)


[허용 집합] news_times_global: rows=9,100, allowed_ids=9,100, col='news_id'

===== train : 파일 로드 =====
- behaviors: download/MINDsmall_train/behaviors.tsv rows=156,965
- Time NaT(파싱 실패) 건수: 0


[train] scanning: 100%|██████████| 156965/156965 [00:03<00:00, 40238.50it/s]



===== train : 요약 (허용집합=news_times_global) =====
원본 행수: 156,965  →  예상 변경 후: 236,344  (Δ +79,379)
- Time이 NaT인 행 수: 0
- 허용집합 기준 -1이 2개 이상인 행 수: 43,077
- 그 행들에서의 -1 총합: 122,456
- 허용집합 기준 -1≥1 인 행 수: 156,965

[-1 개수 분포 (허용집합 기준) 상위]
2개 -1: 25571행
3개 -1: 9263행
4개 -1: 3975행
5개 -1: 1957행
6개 -1: 942행
7개 -1: 515행
8개 -1: 296행
9개 -1: 198행
10개 -1: 117행
11개 -1: 81행
12개 -1: 46행
13개 -1: 38행
14개 -1: 22행
15개 -1: 17행
16개 -1: 10행
... (그 외 13개 bin)

===== dev : 파일 로드 =====
- behaviors: download/MINDsmall_dev/behaviors.tsv rows=73,152
- Time NaT(파싱 실패) 건수: 0


[dev] scanning: 100%|██████████| 73152/73152 [00:01<00:00, 40796.04it/s]


===== dev : 요약 (허용집합=news_times_global) =====
원본 행수: 73,152  →  예상 변경 후: 111,383  (Δ +38,231)
- Time이 NaT인 행 수: 0
- 허용집합 기준 -1이 2개 이상인 행 수: 21,085
- 그 행들에서의 -1 총합: 59,316
- 허용집합 기준 -1≥1 인 행 수: 73,152

[-1 개수 분포 (허용집합 기준) 상위]
2개 -1: 12707행
3개 -1: 4443행
4개 -1: 1911행
5개 -1: 932행
6개 -1: 426행
7개 -1: 268행
8개 -1: 166행
9개 -1: 83행
10개 -1: 61행
11개 -1: 34행
12개 -1: 17행
13개 -1: 11행
14개 -1: 5행
15개 -1: 5행
16개 -1: 6행
... (그 외 6개 bin)


### 뉴스 수명 계산
#### train/dev 분리하여 생성
#### 뉴스 출판 시간: impression Time에서 가장 최초의 시간
#### 뉴스 수명 시간: impression Time에서 +36시간

In [18]:
# A_make_lifespan_tables.py
import os
import pandas as pd
from datetime import timedelta

# =========================================================
# 설정
# =========================================================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 파일 경로 (필요시 수정)
PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'

# 9,100 허용 집합(클릭 고유 뉴스 ID) 파일
PATH_CLICKED_IDS = 'clicked_newsIds_global.csv'   # (1단계 산출물, col: news_id)

# 출력 파일 (split별 lifespan)
OUT_TRAIN_LIFE = 'train_lifespan.csv'   # ['news_id','publish_time','lifespan_end']
OUT_DEV_LIFE   = 'dev_lifespan.csv'


# =========================================================
# 0) 보조 함수
# =========================================================
def build_clicked_set(df):
    """
    df 전체에서 클릭(-1)된 뉴스의 고유 ID 집합을 만든다.
    - Impressions를 공백으로 나눔
    - "newsId-label"에서 label이 '1'인 것만 수집
    - rsplit('-', 1)로 뒤에서 한 번만 분리 (안전)
    """
    s = set()
    for _, row in df.iterrows():
        tokens = str(row['Impressions']).split()
        for tok in tokens:
            if '-' not in tok:
                continue
            nid, lbl = tok.rsplit('-', 1)
            if lbl.strip() == '1':
                s.add(nid.strip())
    return s


def build_lifespan_table(df, allowed_ids):
    """
    split 하나에 대한 수명표 만들기 (허용 집합에 포함된 뉴스만):
      - 각 뉴스ID의 '해당 split에서의 최초 등장 시각'을 publish_time으로
      - lifespan_end = publish_time + 36시간
      - 반환: DataFrame(columns=['news_id','publish_time','lifespan_end'])
    """
    first_time = {}  # dict: news_id -> 가장 이른 datetime

    for _, row in df.iterrows():
        t = row['Time']  # 이 행의 노출 시각
        if pd.isna(t):
            continue
        tokens = str(row['Impressions']).split()
        for token in tokens:
            if '-' not in token:
                continue
            news_id, _ = token.rsplit('-', 1)
            news_id = news_id.strip()

            # ★ 허용 집합(9,100)에 포함된 뉴스만 대상
            if news_id not in allowed_ids:
                continue

            # 최초 등장 시각 갱신
            if news_id not in first_time:
                first_time[news_id] = t
            else:
                if t < first_time[news_id]:
                    first_time[news_id] = t

    # 표로 변환
    rows = []
    for nid, pub_t in first_time.items():
        lifespan_end = pub_t + timedelta(hours=36)
        rows.append((nid, pub_t, lifespan_end))

    life_df = pd.DataFrame(rows, columns=['news_id', 'publish_time', 'lifespan_end'])
    return life_df


# =========================================================
# 1) behaviors 불러오기 (헤더 없음 주의) + 시간 파싱
# =========================================================
train = pd.read_csv(PATH_TRAIN, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
dev   = pd.read_csv(PATH_DEV,   sep='\t', names=BEHAVIOR_COLUMNS, header=None)

train['Time'] = pd.to_datetime(train['Time'], errors='coerce')
dev['Time']   = pd.to_datetime(dev['Time'],   errors='coerce')

# 허용 집합 파일이 없으면 즉석에서 생성 (train+dev 합쳐서 클릭 뉴스 뽑음)
if os.path.exists(PATH_CLICKED_IDS):
    allow_df = pd.read_csv(PATH_CLICKED_IDS)
    allowed_ids = set(str(x).strip() for x in allow_df['news_id'].dropna().tolist())
    print(f"[OK] 허용 집합 로드: {PATH_CLICKED_IDS} (개수={len(allowed_ids)})")
else:
    print(f"[INFO] 허용 집합 파일 없음 → 즉석 생성: {PATH_CLICKED_IDS}")
    both = pd.concat([train, dev], ignore_index=True)
    allowed_ids = build_clicked_set(both)
    pd.DataFrame({'news_id': sorted(allowed_ids)}).to_csv(PATH_CLICKED_IDS, index=False)
    print(f"[OK] 생성 및 저장 완료: {PATH_CLICKED_IDS} (개수={len(allowed_ids)})")

# =========================================================
# 2) split별 수명표 생성 & 저장 (★ train/dev 각각, 그리고 ★ 허용 집합 제한 적용)
# =========================================================
train_life = build_lifespan_table(train, allowed_ids)
dev_life   = build_lifespan_table(dev,   allowed_ids)

train_life.to_csv(OUT_TRAIN_LIFE, index=False)
dev_life.to_csv(OUT_DEV_LIFE,     index=False)

print("완료: 수명표 저장")
print(" -", OUT_TRAIN_LIFE, len(train_life))
print(" -", OUT_DEV_LIFE,   len(dev_life))

# (선택) 각 split에서 허용 집합 중 등장하지 않은 ID 수 체크(모니터링용)
miss_train = len(allowed_ids - set(train_life['news_id']))
miss_dev   = len(allowed_ids - set(dev_life['news_id']))
print(f"허용 ID 중 train에 등장하지 않은 개수: {miss_train}")
print(f"허용 ID 중 dev에 등장하지 않은 개수  : {miss_dev}")


[OK] 허용 집합 로드: clicked_newsIds_global.csv (개수=9100)
완료: 수명표 저장
 - train_lifespan.csv 8069
 - dev_lifespan.csv 3221
허용 ID 중 train에 등장하지 않은 개수: 1031
허용 ID 중 dev에 등장하지 않은 개수  : 5879


## 뉴스 추천 데이터 behaviors 재생성
### train/dev 데이터 분리하여 생성
### 클릭한 뉴스가 한 행에 여러 행이 있는 경우 행 분리
### 클릭하지 않은 뉴스는 9100개로 분리된 뉴스 중에서 해당 TIME에 수명시간이 지나지 않은것 20개 랜덤 할당


In [19]:
# # lifespan을 train dev 나눠서 진행
# import pandas as pd
# import random
# from datetime import timedelta

# random.seed(42)  # 무작위 샘플 재현용 시드

# # =========================================================
# # 설정
# # =========================================================
# BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# # 파일 경로 (필요 시 수정)
# PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
# PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'

# # (A 단계에서 미리 만든 수명표 파일들: ★ 9,100 허용 집합만 포함되어 있어야 함)
# PATH_TRAIN_LIFE = 'train_lifespan.csv'  # ['news_id','publish_time','lifespan_end']
# PATH_DEV_LIFE   = 'dev_lifespan.csv'

# K_NEG = 20  # 음성 개수 (한 행당: 양성 1 + 음성 K_NEG)
# SHUFFLE_IMPRESSIONS = True

# # =========================================================
# # 공용 보조 함수들 (초보자 스타일)
# # =========================================================
# def get_all_positives(imps_str):
#     """
#     Impressions 문자열에서 -1(클릭)인 모든 뉴스ID 리스트를 반환.
#     예: "N1-0 N2-1 N3-1 N4-0" -> ["N2", "N3"]
#     클릭이 하나도 없으면 [] 반환.
#     """
#     result = []
#     if pd.isna(imps_str):
#         return result
#     tokens = str(imps_str).split()
#     for token in tokens:
#         if '-' not in token:
#             continue
#         news_id, label = token.rsplit('-', 1)
#         if label.strip() == '1':  # ★ 꼭 -1 인 것만 양성
#             result.append(news_id.strip())
#     return result

# def floor_to_hour(dt):
#     """
#     datetime을 '정시'로 내림.
#     예: 2020-06-14 13:27 -> 2020-06-14 13:00
#     """
#     if pd.isna(dt):
#         return None
#     return dt.replace(minute=0, second=0, microsecond=0)

# def build_user_clicked(df):
#     """
#     df 전체를 훑어 사용자별 '과거/미래 포함, 한 번이라도 클릭(-1)한 뉴스ID' 집합 생성.
#     반환: dict { user_id: set(news_id) }
#     """
#     user_clicked = {}
#     for _, row in df.iterrows():
#         uid = row['UserID']
#         tokens = str(row['Impressions']).split()
#         for token in tokens:
#             if '-' not in token:
#                 continue
#             news_id, label = token.rsplit('-', 1)
#             if label.strip() == '1':
#                 if uid not in user_clicked:
#                     user_clicked[uid] = set()
#                 user_clicked[uid].add(news_id.strip())
#     return user_clicked

# def build_alive_buckets_and_map(life_df):
#     """
#     시간 버킷(정시 단위) → 그 시간에 '살아있는' 뉴스ID 리스트를 빠르게 얻기 위한 사전 구성.
#     또한 정확한 분 단위 판정을 위해 news_id -> (publish_time, lifespan_end) map도 함께 생성.

#     buckets: dict { hour_dt: [news_id, ...] }
#     life_map: dict { news_id: (publish_time, lifespan_end) }
#     """
#     buckets = {}
#     life_map = {}

#     for _, row in life_df.iterrows():
#         nid   = str(row['news_id'])
#         start = row['publish_time']
#         end   = row['lifespan_end']
#         life_map[nid] = (start, end)

#         if pd.isna(start) or pd.isna(end):
#             continue

#         cur = floor_to_hour(start)
#         # cur 가 end 이전까지만(행 기준 조건: row_time < lifespan_end)
#         while cur < end:
#             if cur not in buckets:
#                 buckets[cur] = []
#             buckets[cur].append(nid)
#             cur = cur + timedelta(hours=1)

#     return buckets, life_map


# # =========================================================
# # split 하나를 처리: 다중 양성 행 → '양성 개수만큼 행 확장' + 음성 샘플링
# #  - ★ 양성도 lifespan(= 허용 집합) 안에 있는 뉴스만 사용하도록 필터링
# #  - 음성은 lifespan 버킷에서 선택(= 허용 집합 내부) + 유저 과거/미래 미클릭 + HISTORY 미포함 + 양성과 비중복
# # =========================================================
# def rebuild_for_split_with_expand(beh_path, life_path, k_neg):
#     # 1) behaviors 불러오기 (헤더 없음)
#     df = pd.read_csv(beh_path, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
#     df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

#     # 2) 수명표 로드 + 버킷/맵 생성
#     life_df = pd.read_csv(life_path, parse_dates=['publish_time','lifespan_end'])
#     alive_buckets, life_map = build_alive_buckets_and_map(life_df)

#     # 3) 사용자별 '클릭했던 뉴스 집합' (해당 split 전체 기준)
#     u_clicked = build_user_clicked(df)

#     # 4) 결과를 담을 '확장된 행' 리스트
#     expanded_rows = []

#     # 5) 각 원본 행을 순회
#     for _, row in df.iterrows():
#         row_time = row['Time']
#         uid      = row['UserID']
#         original = row['Impressions']

#         # (a) 이 행의 모든 양성 뉴스ID 수집
#         pos_list_all = get_all_positives(original)
#         # (a-1) ★ 양성도 허용 집합(= life_map 키) 안의 뉴스만 사용
#         allowed_pos = []
#         for nid in pos_list_all:
#             if nid in life_map:
#                 allowed_pos.append(nid)

#         # (b) 양성이 하나도 없거나 시간 파싱 실패라면 → 원본 그 상태로 1행만 유지
#         if (len(allowed_pos) == 0) or pd.isna(row_time):
#             new_row = {
#                 'ImpressionID': row['ImpressionID'],
#                 'UserID': row['UserID'],
#                 'Time': row['Time'],
#                 'History': row['History'],
#                 'Impressions': row['Impressions'],
#                 'Impressions_new': row['Impressions']  # 그대로 둠(원하면 ""로 비울 수도 있음)
#             }
#             expanded_rows.append(new_row)
#             continue

#         # (c) 양성이 여러 개인 경우 → '양성 1개당 1행'씩 복제 생성
#         hour_key = floor_to_hour(row_time)
#         candidate_ids_bucket = []
#         if (hour_key is not None) and (hour_key in alive_buckets):
#             candidate_ids_bucket = alive_buckets[hour_key][:]
#         # 중복 제거
#         candidate_ids_bucket = list(set(candidate_ids_bucket))

#         # History set 준비
#         history_set = set()
#         if pd.notna(row['History']):
#             for nid in str(row['History']).split():
#                 history_set.add(nid)

#         # pos마다 한 행 생성
#         for pos_news in allowed_pos:
#             # 1) 후보 시작: 버킷에서 꺼낸 리스트 → 정밀 시간조건으로 다시 필터링
#             precise_candidates = []
#             for nid in candidate_ids_bucket:
#                 pub_t, end_t = life_map.get(nid, (None, None))
#                 if (pub_t is None) or (end_t is None):
#                     continue
#                 if (pub_t <= row_time) and (row_time < end_t):
#                     precise_candidates.append(nid)

#             # 2) 사용자 과거/미래 클릭 제외
#             clicked_set = u_clicked.get(uid, set())
#             tmp = []
#             for nid in precise_candidates:
#                 if nid not in clicked_set:
#                     tmp.append(nid)
#             precise_candidates = tmp

#             # 3) History 제외
#             tmp = []
#             for nid in precise_candidates:
#                 if nid not in history_set:
#                     tmp.append(nid)
#             precise_candidates = tmp

#             # 4) 양성과 중복 금지
#             tmp = []
#             for nid in precise_candidates:
#                 if nid != pos_news:
#                     tmp.append(nid)
#             precise_candidates = tmp

#             # 5) 음성 무작위 추출 (부족하면 있는 만큼만)
#             if len(precise_candidates) >= k_neg:
#                 neg_news = random.sample(precise_candidates, k_neg)
#             else:
#                 neg_news = precise_candidates

#             # 6) 최종 Impressions 문자열 (양성 -1, 음성 -0)
#             parts = [f"{pos_news}-1"]
#             for nid in neg_news:
#                 parts.append(f"{nid}-0")
#             if SHUFFLE_IMPRESSIONS:
#                 random.shuffle(parts)
#             new_imps_str = " ".join(parts)

#             # 7) 원본 행을 복제하되, Impressions_new만 다르게 설정
#             new_row = {
#                 'ImpressionID': row['ImpressionID'],   # ★ 동일 ID 유지(필요시 새 ID 부여 가능)
#                 'UserID': row['UserID'],
#                 'Time': row['Time'],
#                 'History': row['History'],
#                 'Impressions': row['Impressions'],
#                 'Impressions_new': new_imps_str
#             }
#             expanded_rows.append(new_row)

#     # 6) 확장된 행들로 DataFrame 생성 (원본 5컬럼 + Impressions_new)
#     out_df = pd.DataFrame(expanded_rows, columns=BEHAVIOR_COLUMNS + ['Impressions_new'])
#     return df, out_df   # 원본 df도 함께 반환하여 개수 비교에 사용


# # =========================================================
# # 실행: train/dev 각각 처리 + 행 개수 비교 출력
# # =========================================================
# # train
# train_orig_df, train_final = rebuild_for_split_with_expand(PATH_TRAIN, PATH_TRAIN_LIFE, K_NEG)
# # dev
# dev_orig_df,   dev_final   = rebuild_for_split_with_expand(PATH_DEV,   PATH_DEV_LIFE,   K_NEG)

# # --- 결과 미리보기
# print(train_final.head())
# print(dev_final.head())

# # --- 행 개수 비교 출력
# orig_train_rows = len(train_orig_df)
# orig_dev_rows   = len(dev_orig_df)
# new_train_rows  = len(train_final)
# new_dev_rows    = len(dev_final)

# print("\n===== 행 개수 비교 =====")
# print(f"train  | 원본: {orig_train_rows:,}  →  변경 후: {new_train_rows:,}  (Δ {new_train_rows - orig_train_rows:+,})")
# print(f"dev    | 원본: {orig_dev_rows:,}    →  변경 후: {new_dev_rows:,}    (Δ {new_dev_rows - orig_dev_rows:+,})")

# # (선택) 저장
# train_final.to_csv('train_expanded_global.tsv', sep='\t', index=False)
# dev_final.to_csv('dev_expanded_global.tsv',   sep='\t', index=False)

# # (선택) 학습 포맷(5컬럼)으로 쓰려면, Impressions ← Impressions_new 교체 + 5컬럼만:
# train_out = train_final.copy()
# train_out['Impressions'] = train_out['Impressions_new']
# train_out = train_out[BEHAVIOR_COLUMNS]
# train_out.to_csv('train_rebuilt_expanded_global.tsv', sep='\t', header=False, index=False)

# dev_out = dev_final.copy()
# dev_out['Impressions'] = dev_out['Impressions_new']
# dev_out = dev_out[BEHAVIOR_COLUMNS]
# dev_out.to_csv('dev_rebuilt_expanded_global.tsv', sep='\t', header=False, index=False)


   ImpressionID  UserID                Time  \
0             1  U13740 2019-11-11 09:05:58   
1             2  U91836 2019-11-12 18:11:30   
2             3  U73700 2019-11-14 07:01:48   
3             4  U34670 2019-11-11 05:28:05   
4             5   U8125 2019-11-12 16:11:21   

                                             History  \
0  N55189 N42782 N34694 N45794 N18445 N63302 N104...   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129...   
2  N10732 N25792 N7563 N21087 N41087 N5445 N60384...   
3  N45729 N2203 N871 N53880 N41375 N43142 N33013 ...   
4                        N10078 N56514 N14904 N33740   

                                         Impressions  \
0                                  N55689-1 N35729-0   
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...   
2  N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...   
3                N35729-0 N33632-0 N49685-1 N27581-0   
4  N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...   

                                     Impres

In [20]:
# lifespan이 global

import os
import pandas as pd
import random
from datetime import timedelta

random.seed(42)  # 재현성

# =========================
# 설정
# =========================
BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
PATH_TRAIN = 'download/MINDsmall_train/behaviors.tsv'
PATH_DEV   = 'download/MINDsmall_dev/behaviors.tsv'

# 전역 허용집합 / 전역 lifespan 파일
PATH_CLICKED_GLOBAL = 'clicked_newsIds_global.csv'   # col: news_id
PATH_LIFE_GLOBAL    = 'news_times_global.csv'        # col: news_id, publish_time, lifespan_end

K_NEG = 20
SHUFFLE_IMPRESSIONS = True


# =========================
# 보조 함수
# =========================
def get_all_positives(imps_str):
    out = []
    if pd.isna(imps_str):
        return out
    for tok in str(imps_str).split():
        if '-' not in tok:
            continue
        nid, lab = tok.rsplit('-', 1)
        if lab.strip() == '1':
            out.append(nid.strip())
    return out

def floor_to_hour(dt):
    if pd.isna(dt):
        return None
    return dt.replace(minute=0, second=0, microsecond=0)

def build_user_clicked(df):
    m = {}
    for _, row in df.iterrows():
        uid = row['UserID']
        for tok in str(row['Impressions']).split():
            if '-' not in tok:
                continue
            nid, lab = tok.rsplit('-', 1)
            if lab.strip() == '1':
                m.setdefault(uid, set()).add(nid.strip())
    return m

def build_clicked_set(df):
    s = set()
    for _, row in df.iterrows():
        for tok in str(row['Impressions']).split():
            if '-' not in tok:
                continue
            nid, lab = tok.rsplit('-', 1)
            if lab.strip() == '1':
                s.add(nid.strip())
    return s

def build_global_lifespan(behaviors_df, allowed_ids):
    # 각 뉴스의 "전역 최초 노출 시각"을 publish_time으로, +36h
    first_time = {}
    for _, row in behaviors_df.iterrows():
        t = row['Time']
        if pd.isna(t):
            continue
        for tok in str(row['Impressions']).split():
            if '-' not in tok:
                continue
            nid, _ = tok.rsplit('-', 1)
            nid = nid.strip()
            if nid not in allowed_ids:
                continue
            if nid not in first_time or t < first_time[nid]:
                first_time[nid] = t
    rows = []
    for nid, pub in first_time.items():
        rows.append((nid, pub, pub + timedelta(hours=36)))
    life = pd.DataFrame(rows, columns=['news_id','publish_time','lifespan_end'])
    life = life.sort_values('publish_time').reset_index(drop=True)
    return life

def build_alive_buckets_and_map(life_df):
    buckets = {}
    life_map = {}
    for _, r in life_df.iterrows():
        nid   = str(r['news_id']).strip()
        start = r['publish_time']
        end   = r['lifespan_end']
        life_map[nid] = (start, end)
        if pd.isna(start) or pd.isna(end):
            continue
        cur = floor_to_hour(start)
        while cur < end:
            buckets.setdefault(cur, []).append(nid)
            cur = cur + timedelta(hours=1)
    return buckets, life_map


# =========================
# 전역 허용집합 / 전역 lifespan 준비
# =========================
# train/dev 로드 + 시간 파싱
train = pd.read_csv(PATH_TRAIN, sep='\t', names=BEHAVIOR_COLUMNS, header=None)
dev   = pd.read_csv(PATH_DEV,   sep='\t', names=BEHAVIOR_COLUMNS, header=None)
train['Time'] = pd.to_datetime(train['Time'], errors='coerce')
dev['Time']   = pd.to_datetime(dev['Time'],   errors='coerce')

both = pd.concat([train, dev], ignore_index=True)

# 전역 허용집합
if os.path.exists(PATH_CLICKED_GLOBAL):
    allow_df = pd.read_csv(PATH_CLICKED_GLOBAL)
    allowed_ids = set(allow_df['news_id'].astype(str).str.strip())
else:
    allowed_ids = build_clicked_set(both)
    pd.DataFrame({'news_id': sorted(allowed_ids)}).to_csv(PATH_CLICKED_GLOBAL, index=False)

# 전역 lifespan
if os.path.exists(PATH_LIFE_GLOBAL):
    life_global = pd.read_csv(PATH_LIFE_GLOBAL, parse_dates=['publish_time','lifespan_end'])
else:
    life_global = build_global_lifespan(both, allowed_ids)
    life_global.to_csv(PATH_LIFE_GLOBAL, index=False)

# 전역 버킷/맵 (train/dev 모두 이걸 사용)
alive_buckets_global, life_map_global = build_alive_buckets_and_map(life_global)


# =========================
# 재구성 함수 (전역 lifespan 사용)
# =========================
def rebuild_with_global_life(df_split, k_neg):
    u_clicked = build_user_clicked(df_split)
    expanded_rows = []

    for _, row in df_split.iterrows():
        row_time = row['Time']
        uid      = row['UserID']
        original = row['Impressions']

        # 양성 후보: 전역 허용집합(= life_map_global 키) 안에서만
        pos_all = get_all_positives(original)
        allowed_pos = [nid for nid in pos_all if nid in life_map_global]

        # 양성 없거나 Time NaT -> 원본 유지
        if (len(allowed_pos) == 0) or pd.isna(row_time):
            expanded_rows.append({
                'ImpressionID': row['ImpressionID'],
                'UserID': row['UserID'],
                'Time': row['Time'],
                'History': row['History'],
                'Impressions': row['Impressions'],
                'Impressions_new': row['Impressions']
            })
            continue

        # 시간 버킷 후보(전역)
        hour_key = floor_to_hour(row_time)
        bucket = list(set(alive_buckets_global.get(hour_key, [])))  # 중복 제거

        # HISTORY set
        history_set = set(str(row['History']).split()) if pd.notna(row['History']) else set()

        for pos_news in allowed_pos:
            # 음성 후보: alive(정밀) → 미클릭 → 미히스토리 → 양성과 비중복
            precise = []
            for nid in bucket:
                pub, end = life_map_global.get(nid, (None, None))
                if (pub is not None) and (end is not None) and (pub <= row_time < end):
                    precise.append(nid)

            clicked_set = u_clicked.get(uid, set())
            tmp = []
            for nid in precise:
                if (nid not in clicked_set) and (nid not in history_set) and (nid != pos_news):
                    tmp.append(nid)
            precise = tmp

            if len(precise) >= k_neg:
                neg_news = random.sample(precise, k_neg)
            else:
                neg_news = precise

            parts = [f"{pos_news}-1"] + [f"{nid}-0" for nid in neg_news]
            if SHUFFLE_IMPRESSIONS:
                random.shuffle(parts)
            new_imps = " ".join(parts)

            expanded_rows.append({
                'ImpressionID': row['ImpressionID'],
                'UserID': row['UserID'],
                'Time': row['Time'],
                'History': row['History'],
                'Impressions': row['Impressions'],
                'Impressions_new': new_imps
            })

    out = pd.DataFrame(expanded_rows, columns=BEHAVIOR_COLUMNS + ['Impressions_new'])
    return out


# =========================
# 실행: train/dev 각각 전역 lifespan으로 재구성 + 행수 비교
# =========================
train_final = rebuild_with_global_life(train, K_NEG)
dev_final   = rebuild_with_global_life(dev,   K_NEG)

for name, df in (("train", train_final), ("dev", dev_final)):
    # 원본 ID를 보존(추적용)
    df.insert(0, 'OriginalImpressionID', df['ImpressionID'])
    # 새 ID 부여: 1..len(df)
    df['ImpressionID'] = range(1, len(df) + 1)

print(train_final.head())
print(dev_final.head())

print("\n===== 행 개수 비교 =====")
print(f"train | 원본: {len(train):,}  →  변경 후: {len(train_final):,}  (Δ {len(train_final) - len(train):+,})")
print(f"dev   | 원본: {len(dev):,}    →  변경 후: {len(dev_final):,}    (Δ {len(dev_final) - len(dev):+,})")

# 저장 (필요 시)
train_final.to_csv('train_expanded_global.tsv', sep='\t', index=False)
dev_final.to_csv('dev_expanded_global.tsv',   sep='\t', index=False)

# 2) 학습용(5컬럼)으로 저장: Impressions <- Impressions_new 치환 후 BEHAVIOR_COLUMNS만
def to_training_5col(df, out_path):
    out = df.copy()
    out['Impressions'] = out['Impressions_new']
    out = out[['ImpressionID','UserID','Time','History','Impressions']]
    # 모델 호환을 위해 header=False
    out.to_csv(out_path, sep='\t', header=False, index=False)

to_training_5col(train_final, 'train_rebuilt_expanded_global.tsv')
to_training_5col(dev_final,   'dev_rebuilt_expanded_global.tsv')



   OriginalImpressionID  ImpressionID  UserID                Time  \
0                     1             1  U13740 2019-11-11 09:05:58   
1                     2             2  U91836 2019-11-12 18:11:30   
2                     3             3  U73700 2019-11-14 07:01:48   
3                     4             4  U34670 2019-11-11 05:28:05   
4                     5             5   U8125 2019-11-12 16:11:21   

                                             History  \
0  N55189 N42782 N34694 N45794 N18445 N63302 N104...   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129...   
2  N10732 N25792 N7563 N21087 N41087 N5445 N60384...   
3  N45729 N2203 N871 N53880 N41375 N43142 N33013 ...   
4                        N10078 N56514 N14904 N33740   

                                         Impressions  \
0                                  N55689-1 N35729-0   
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...   
2  N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...   
3                N35729-

# 검증하기

In [21]:
import pandas as pd

BEHAVIOR_COLUMNS = ['ImpressionID','UserID','Time','History','Impressions']
K_NEG = 20

# -------------------------------------------
# 유틸: impressions 문자열 파싱 (초보자 안전 버전)
# -------------------------------------------
def parse_imps(imps_str):
    """
    'newsid-label' 토큰들로 분할하여 [(nid, lab), ...] 리스트 반환
    lab은 '0' 또는 '1'만 유효로 간주
    """
    out = []
    if pd.isna(imps_str):
        return out
    for tok in str(imps_str).split():
        if '-' not in tok:
            continue
        nid, lab = tok.rsplit('-', 1)
        nid = nid.strip()
        lab = lab.strip()
        if lab in ('0','1'):
            out.append((nid, lab))
    return out

# -------------------------------------------
# 핵심 검증 함수 (split 하나)
# -------------------------------------------
def validate_split(final_df, life_path, split_name="split", k_neg=20, max_show=5):
    """
    final_df: 최종 산출 DF (원본 5컬럼 + Impressions_new)
    life_path: lifespan csv 경로 (['news_id','publish_time','lifespan_end'])
    """
    print(f"\n===== 검증 시작: {split_name} =====")

    # 0) 준비: 시간 파싱, 허용 집합, life_map
    df = final_df.copy()
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

    life_df = pd.read_csv(life_path, parse_dates=['publish_time','lifespan_end'])
    allowed = set(life_df['news_id'].astype(str).str.strip())
    life_map = {}
    for _, r in life_df.iterrows():
        life_map[str(r['news_id']).strip()] = (r['publish_time'], r['lifespan_end'])

    # 1) 통계 카운터
    total_rows = len(df)
    viol_not_allowed = 0
    viol_not_allowed_examples = []

    viol_poscount = 0
    viol_poscount_examples = []

    viol_negcount = 0
    viol_negcount_examples = []

    viol_neg_lifespan = 0
    viol_neg_lifespan_examples = []

    # (옵션) 양성 alive 체크
    viol_pos_lifespan = 0
    viol_pos_lifespan_examples = []

    # (보너스) 중복 뉴스ID 체크
    viol_dup_ids = 0
    viol_dup_ids_examples = []

    # 2) 행별 점검
    for idx, row in df.iterrows():
        t = row['Time']
        tokens = parse_imps(row['Impressions_new'])

        # 2-1) 허용 집합(= 9,100) 내부인지
        news_ids = [nid for nid, _ in tokens]
        not_allowed = [nid for nid in news_ids if nid not in allowed]
        if len(not_allowed) > 0:
            viol_not_allowed += 1
            if len(viol_not_allowed_examples) < max_show:
                viol_not_allowed_examples.append((idx, not_allowed[:5]))

        # 2-2) 양성/음성 개수
        pos_cnt = sum(1 for _, lab in tokens if lab == '1')
        neg_cnt = sum(1 for _, lab in tokens if lab == '0')

        if pos_cnt != 1:
            viol_poscount += 1
            if len(viol_poscount_examples) < max_show:
                viol_poscount_examples.append((idx, pos_cnt, row['Impressions_new']))

        # 후보 부족이면 20 미만일 수 있음 → 그래도 통계로 확인
        # (원칙: 정상 행은 neg_cnt == k_neg)
        if (pos_cnt == 1) and (neg_cnt != k_neg):
            viol_negcount += 1
            if len(viol_negcount_examples) < max_show:
                viol_negcount_examples.append((idx, neg_cnt, row['Impressions_new']))

        # 2-3) lifespan 조건(음성): publish ≤ Time < lifespan_end
        #  (t가 NaT면 이미 확장 안 했을 가능성이 큼. 그래도 방어적으로 검사)
        bad_neg = False
        for nid, lab in tokens:
            if lab != '0':
                continue
            if nid not in life_map or pd.isna(t):
                bad_neg = True
                break
            pub, end = life_map[nid]
            if pd.isna(pub) or pd.isna(end) or not (pub <= t < end):
                bad_neg = True
                break
        if bad_neg:
            viol_neg_lifespan += 1
            if len(viol_neg_lifespan_examples) < max_show:
                viol_neg_lifespan_examples.append((idx, row['Time'], row['Impressions_new']))

        # 2-4) (옵션) 양성 alive 확인
        bad_pos = False
        for nid, lab in tokens:
            if lab != '1':
                continue
            if nid not in life_map or pd.isna(t):
                bad_pos = True
                break
            pub, end = life_map[nid]
            if pd.isna(pub) or pd.isna(end) or not (pub <= t < end):
                bad_pos = True
                break
        if bad_pos:
            viol_pos_lifespan += 1
            if len(viol_pos_lifespan_examples) < max_show:
                viol_pos_lifespan_examples.append((idx, row['Time'], row['Impressions_new']))

        # 2-5) (보너스) 동일 뉴스ID 중복 여부
        if len(news_ids) != len(set(news_ids)):
            viol_dup_ids += 1
            if len(viol_dup_ids_examples) < max_show:
                viol_dup_ids_examples.append((idx, row['Impressions_new']))

    # 3) 요약 출력
    print(f"- 총 행수: {total_rows:,}")
    print(f"[조건1] 허용 집합(9100) 밖 뉴스 포함 행: {viol_not_allowed:,}")
    if viol_not_allowed_examples:
        print("  예시:", viol_not_allowed_examples[:max_show])

    print(f"[조건2-양성] 양성(-1) 개수가 1이 아닌 행: {viol_poscount:,}")
    if viol_poscount_examples:
        print("  예시( idx, pos_cnt, row ):")
        for ex in viol_poscount_examples:
            print("  ", ex)

    print(f"[조건2-음성] 음성(-0) 개수가 {k_neg}이 아닌 행(양성=1인 경우): {viol_negcount:,}")
    if viol_negcount_examples:
        print("  예시( idx, neg_cnt, row ):")
        for ex in viol_negcount_examples:
            print("  ", ex)

    print(f"[조건3-음성수명] Time에 alive가 아닌 음성을 포함한 행: {viol_neg_lifespan:,}")
    if viol_neg_lifespan_examples:
        print("  예시( idx, Time, row ):")
        for ex in viol_neg_lifespan_examples:
            print("  ", ex)

    print(f"[옵션-양성수명] Time에 alive가 아닌 양성을 포함한 행: {viol_pos_lifespan:,}")
    if viol_pos_lifespan_examples:
        print("  예시( idx, Time, row ):")
        for ex in viol_pos_lifespan_examples:
            print("  ", ex)

    print(f"[보너스] 동일 뉴스ID가 중복으로 들어간 행: {viol_dup_ids:,}")
    if viol_dup_ids_examples:
        print("  예시:", viol_dup_ids_examples[:max_show])

    print(f"===== 검증 종료: {split_name} =====\n")


In [22]:
train_final = pd.read_csv('train_expanded_global.tsv', sep='\t')
dev_final   = pd.read_csv('dev_expanded_global.tsv',   sep='\t')
validate_split(train_final, 'news_times_global.csv', split_name='train', k_neg=20)
validate_split(dev_final, 'news_times_global.csv', split_name='dev', k_neg=20)
#validate_split(train_final, 'train_lifespan.csv', split_name='train', k_neg=20)
#validate_split(dev_final,   'dev_lifespan.csv',   split_name='dev',   k_neg=20)



===== 검증 시작: train =====
- 총 행수: 236,344
[조건1] 허용 집합(9100) 밖 뉴스 포함 행: 0
[조건2-양성] 양성(-1) 개수가 1이 아닌 행: 0
[조건2-음성] 음성(-0) 개수가 20이 아닌 행(양성=1인 경우): 0
[조건3-음성수명] Time에 alive가 아닌 음성을 포함한 행: 0
[옵션-양성수명] Time에 alive가 아닌 양성을 포함한 행: 26,796
  예시( idx, Time, row ):
   (11, Timestamp('2019-11-14 08:38:04'), 'N13553-0 N1257-0 N58620-0 N1160-0 N59073-0 N54368-0 N28072-0 N10480-0 N11204-0 N52945-0 N57248-0 N36793-0 N21535-0 N61622-1 N31893-0 N6937-0 N64305-0 N59138-0 N60986-0 N47380-0 N50012-0')
   (35, Timestamp('2019-11-12 08:22:23'), 'N11758-0 N39001-0 N31978-1 N30809-0 N51990-0 N5146-0 N51523-0 N50791-0 N112-0 N14032-0 N42088-0 N11269-0 N43-0 N55355-0 N22829-0 N6953-0 N25748-0 N45456-0 N13902-0 N9322-0 N54993-0')
   (42, Timestamp('2019-11-11 11:09:14'), 'N38516-0 N40574-0 N47303-0 N44446-0 N64542-0 N18319-0 N41881-1 N33069-0 N11658-0 N55261-0 N37833-0 N26728-0 N18613-0 N18660-0 N40991-0 N23422-0 N8859-0 N53300-0 N9866-0 N38433-0 N54232-0')
   (51, Timestamp('2019-11-11 07:06:30'), 'N49240-0 N4342